In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

df= pd.read_csv('/kaggle/input/bigmart-sales-data/Train.csv')
df_test = pd.read_csv('/kaggle/input/bigmart-sales-data/Test.csv')

df.head()

df_test.head()

In [ ]:
df.isnull().any()

df_test.isnull().any()


In [ ]:
df.isnull().sum()

In [ ]:
df['Item_Weight'].value_counts()

In [ ]:
df['Outlet_Size'].value_counts()

In [ ]:
df['Item_Weight'].fillna((df['Item_Weight'].mean()), inplace=True)
df_test['Item_Weight'].fillna((df_test['Item_Weight'].mean()), inplace=True)

In [ ]:
df['Outlet_Size'] = df['Outlet_Size'].fillna(df['Outlet_Size'].mode()[0])
df_test['Outlet_Size'] = df_test['Outlet_Size'].fillna(df_test['Outlet_Size'].mode()[0])

In [ ]:
df.isnull().sum()
df_test.isnull().sum()

In [ ]:
df['Item_Fat_Content'].value_counts()

In [ ]:
df['Item_Visibility'].value_counts()

In [ ]:
df['Outlet_Establishment_Year'].value_counts()

In [ ]:
df['Outlet_Location_Type'].value_counts()

In [ ]:
cat_cols =['Item_Fat_Content','Item_Type','Outlet_Establishment_Year','Outlet_Size','Outlet_Location_Type','Outlet_Type']

from sklearn.preprocessing import OrdinalEncoder

enc=OrdinalEncoder()

df[cat_cols]=enc.fit_transform(df[cat_cols])

df.head()

In [ ]:
df.drop(['Item_Identifier','Outlet_Identifier'],axis=1,inplace=True)




In [ ]:
from sklearn.preprocessing import StandardScaler

stds=StandardScaler()

X=df.drop('Item_MRP',axis=1)
y=df['Item_MRP']

X=stds.fit_transform(X)

X

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val = train_test_split(X,y,train_size=0.5,shuffle=True)

In [ ]:
df_test.drop(['Item_Identifier','Outlet_Identifier'],axis=1,inplace=True)

cat_cols =['Item_Fat_Content','Item_Type','Outlet_Establishment_Year','Outlet_Size','Outlet_Location_Type','Outlet_Type']






In [ ]:
from sklearn.preprocessing import OrdinalEncoder

enc=OrdinalEncoder()

df_test[cat_cols]=enc.fit_transform(df_test[cat_cols])

df_test.head()

X_test=df_test

In [ ]:
X_test=stds.transform(X_test)

X_test

In [ ]:
from xgboost import XGBRegressor

xgb=XGBRegressor()

xgb.fit(X_train,y_train)

y_pred=xgb.predict(X_val)

In [ ]:
import optuna

from sklearn.metrics import mean_squared_error


def objective(trial):
           
    # To select which parameters to optimize, please look at the XGBoost documentation:
    # https://xgboost.readthedocs.io/en/latest/parameter.html
    param = {
        'tree_method':'gpu_hist',  # Use GPU acceleration
        'lambda': trial.suggest_loguniform(
            'lambda', 1e-3, 10.0
        ),
        'alpha': trial.suggest_loguniform(
            'alpha', 1e-3, 10.0
        ),
        'colsample_bytree': trial.suggest_categorical(
            'colsample_bytree', [0.5,0.6,0.7,0.8,0.9,1.0]
        ),
        'subsample': trial.suggest_categorical(
            'subsample', [0.6,0.7,0.8,1.0]
        ),
        'learning_rate': trial.suggest_categorical(
            'learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02]
        ),
        'n_estimators': trial.suggest_categorical(
            "n_estimators", [150, 200, 300, 3000]
        ),
        'max_depth': trial.suggest_categorical(
            'max_depth', [4,5,7,9,11,13,15,17]
        ),
        'random_state': 42,
        'min_child_weight': trial.suggest_int(
            'min_child_weight', 1, 300
        ),
    }
    model = XGBRegressor(**param)  
    
    model.fit(X_train,y_train,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(X_val)
    
    rmse = mean_squared_error(y_val, preds,squared=False)
    
    return rmse



study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)



In [ ]:
len(y_pred)

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_pred,y_val,squared=False)

In [ ]:
df.corr()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from numpy import hstack

params ={'lambda': 1.1257181656029478, 'alpha': 1.1803723311576444, 'colsample_bytree': 0.7, 'subsample': 1.0, 'learning_rate': 0.014, 'n_estimators': 3000, 'max_depth': 17, 'min_child_weight': 1,'tree_method':'gpu_hist'}

def get_models():
    models = list()
    models.append(('xgb', XGBRegressor(**params)))
    models.append(('rf', RandomForestRegressor()))

    return models

In [ ]:
def fit_ensemble(models, X_train, X_val, y_train, y_val):
       
    meta_X = list()
    for name, model in models:
        # fit in training set
        model.fit(X_train, y_train)
        # predict on hold out set
        yhat = model.predict(X_val)
    # reshape predictions into a matrix with one column
        yhat = yhat.reshape(len(yhat), 1)
#store predictions as input for blending
        meta_X.append(yhat)
# create 2d array from predictions, each set is an input feature
    meta_X = hstack(meta_X)
# define blending model
    blender = LinearRegression()
# fit on predictions from base models
    blender.fit(meta_X, y_val)
    
    return blender

In [ ]:
def predict_ensemble(models, blender, X_test):
    meta_X = list()
    for name, model in models:
        yhat = model.predict(X_test)
        yhat = yhat.reshape(len(yhat), 1)
        meta_X.append(yhat)
    meta_X = hstack(meta_X)
    return blender.predict(meta_X)
   

In [ ]:
models = get_models()
# train the blending ensemble
blender = fit_ensemble(models, X_train, X_val, y_train, y_val)
# make predictions on test set
yhat = predict_ensemble(models, blender, X_test)

yhat
# evaluate predictions


In [ ]:
len(yhat)

In [ ]:
df.head()

In [ ]:
df_y_test_pred=pd.DataFrame({'Target':yhat})

df_y_test_pred.to_csv("submission.csv",index=False)